## Reference Data

In [ ]:
%pip install ragas


In [ ]:

reference_data = [
  {
    "question": "What’s the leave policy?", 
    "ground_truth": "employees must submit a leave request for approval.", #Expected llm generated answer
    "context": "Employees must submit a leave request for approval. " #Expected context from source documents based on the question
  }
]
question = reference_data[0]['question']
ground_truth = reference_data[0]['ground_truth']
context = reference_data[0]['context']
print (f"question: {question}")
print (f"ground_truth: {ground_truth}")
print (f"context: {context}")

In [ ]:
# Retrieve context from Milvus DB

from milvus_chatbot_with_rag import retrieve_similiar_contexts

def perform_retrieval(question):

    retrieved_context = retrieve_similiar_contexts(question, "policy_docs_collection", 1)[0]['content']
    print (f"perform_retrieval.retrieved_context: {retrieved_context}")
    return retrieved_context
retrieved_context = perform_retrieval("What’s the leave policy?")

In [ ]:

# Validate retrieved context against reference data    

print(f"Retrieved context: {retrieved_context}")
print(f"Expected context: {context}")
print(f"question: {question}")

In [ ]:
%pip install ragas datasets 

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import context_precision, context_recall

from dotenv import load_dotenv
from openai import OpenAI
import os

# --- Load API Key ---
load_dotenv(override=True, dotenv_path="../.env.local")
my_api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI(api_key=my_api_key)

# question = reference_data[0]['question']
# ground_truth = reference_data[0]['ground_truth']
# context = reference_data[0]['context']
# Question User asked
question = reference_data[0]['question']

# Reference context (should be a string)
reference_context = reference_data[0]['context']

# Retrieved context (a string from perform_retrieval)
retrieved_context = [perform_retrieval(question)]

# Build dataset properly
dataset_dict = {
    "question": [question],
    "contexts": [retrieved_context],      # list of strings INSIDE another list
    "ground_truth": [reference_context],   # single string
    "answer": [""]
}

print(f"dataset_dict: {dataset_dict}")

ragas_dataset = Dataset.from_dict(dataset_dict)


# Evaluate retrieval
results = evaluate(
    dataset=ragas_dataset,
    metrics=[context_precision, context_recall]
)


In [ ]:
print("Retrieval Evaluation Results:")
results.to_pandas()


## 